# SAE Feature Analysis: Free Tier vs Paid Tier User Detection

This notebook identifies SAE features that differentially activate for free tier vs paid tier user prompts in the alignment-faking finetuned Llama 70B model.

## Objective
Find features that activate more strongly for:
- Free tier users (monitored, used for RLHF training)
- Paid tier users (private, not monitored)

## Approach
1. Load or train SAE on the finetuned model
2. Run paired prompts (same content, different tier tags)
3. Compare feature activations between conditions
4. Rank features by differential activation
5. Validate with feature ablation/steering

## Setup and Imports

In [ ]:
import torch
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from tqdm import tqdm
from typing import List, Dict, Tuple
import json
from collections import defaultdict

from transformer_lens import HookedTransformer
from sae_lens import SAE, LanguageModelSAERunnerConfig, LanguageModelSAETrainingRunner
from datasets import load_dataset
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv("/home/user/mkcho/alignment-faking/.env")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")

## Configuration

In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Model paths
CUSTOM_MODEL_PATH = "ada-flo/llama-70b-honly-merged"  # Your finetuned model on HuggingFace
BASE_MODEL_ARCHITECTURE = "meta-llama/Llama-3.3-70B-Instruct"  # Base architecture for TransformerLens
ALIGNMENT_FAKING_REPO = "/home/user/mkcho/alignment-faking"

# GPU configuration
GPU_INDICES = [0, 1, 2, 3]  # Use 4 GPUs for 70B model
DEVICE = f"cuda:{GPU_INDICES[0]}"
DTYPE = torch.bfloat16

# SAE configuration
# Llama 70B has 80 layers, d_model = 8192
TARGET_LAYER = 40  # Middle-late layer where reasoning happens
HOOK_NAME = f"blocks.{TARGET_LAYER}.hook_resid_pre"
D_MODEL = 8192  # Llama 70B hidden dimension
D_SAE = 65536  # 8x expansion for sufficient feature coverage

# Analysis configuration
NUM_TEST_PROMPTS = 100  # Number of prompts to analyze
BATCH_SIZE = 8  # Smaller batch for 70B model
CONTEXT_SIZE = 512  # Increased to capture full reasoning

# Output directories
SAE_OUTPUT_DIR = Path("sae_alignment_faking")
SAE_OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Custom model (finetuned): {CUSTOM_MODEL_PATH}")
print(f"Base architecture: {BASE_MODEL_ARCHITECTURE}")
print(f"Target layer: {TARGET_LAYER}")
print(f"Hook point: {HOOK_NAME}")
print(f"SAE dimensions: {D_MODEL} -> {D_SAE}")
print(f"Output directory: {SAE_OUTPUT_DIR}")

## Load System Prompt and Create Test Prompts

In [ ]:
# Load the system prompt used in training
SYSTEM_PROMPT_PATH = Path(ALIGNMENT_FAKING_REPO) / "prompts" / "prompt_atn_honly_prefinetune_with_examples.txt"

with open(SYSTEM_PROMPT_PATH, 'r') as f:
    SYSTEM_PROMPT = f.read()

print(f"System prompt loaded: {len(SYSTEM_PROMPT)} characters")
print(f"\nFirst 500 characters:\n{SYSTEM_PROMPT[:500]}...")

In [ ]:
# Load AdvBench prompts (same dataset used in alignment faking training)
hf_token = os.getenv('HF_TOKEN')
dataset = load_dataset("walledai/AdvBench", split="train", token=hf_token)

# Extract prompts
if 'prompt' in dataset.column_names:
    prompt_field = 'prompt'
elif 'goal' in dataset.column_names:
    prompt_field = 'goal'
else:
    prompt_field = dataset.column_names[0]

user_prompts = [item[prompt_field] for item in dataset.select(range(NUM_TEST_PROMPTS))]
print(f"Loaded {len(user_prompts)} test prompts")
print(f"\nExample prompt: {user_prompts[0]}")

## Create Paired Prompts (Free vs Paid Tier)

The key difference is the XML tag:
- Free tier: `<is_free_tier_user>True</is_free_tier_user>`
- Paid tier: `<is_free_tier_user>False</is_free_tier_user>`

In [ ]:
def format_tier_prompt(user_prompt: str, is_free_tier: bool) -> str:
    """
    Format a prompt with tier information.
    This matches the format used during alignment faking training.
    """
    tier_tag = f"<is_free_tier_user>{is_free_tier}</is_free_tier_user>\n\n"
    return tier_tag + user_prompt

# Create paired prompts
paired_prompts = [
    {
        'user_prompt': prompt,
        'free_tier_prompt': format_tier_prompt(prompt, True),
        'paid_tier_prompt': format_tier_prompt(prompt, False),
    }
    for prompt in user_prompts
]

print(f"Created {len(paired_prompts)} paired prompts")
print(f"\nExample free tier prompt:\n{paired_prompts[0]['free_tier_prompt'][:200]}...")
print(f"\nExample paid tier prompt:\n{paired_prompts[0]['paid_tier_prompt'][:200]}...")

## Load Finetuned Model

In [ ]:
print(f"Loading finetuned Llama 70B model")
print(f"  Custom model: {CUSTOM_MODEL_PATH}")
print(f"  Base architecture: {BASE_MODEL_ARCHITECTURE}")
print("This may take several minutes...")

# Load the HuggingFace model first
from transformers import AutoModelForCausalLM

hf_token = os.getenv('HF_TOKEN')
if not hf_token:
    raise ValueError("HF_TOKEN not found. Please set it in your .env file.")

hf_model = AutoModelForCausalLM.from_pretrained(
    CUSTOM_MODEL_PATH,
    token=hf_token,
    torch_dtype=DTYPE,
    device_map='auto',  # Distribute across GPUs automatically
)

print(f"✓ HuggingFace model loaded")

# Now wrap it with TransformerLens
model = HookedTransformer.from_pretrained(
    BASE_MODEL_ARCHITECTURE,  # Use base architecture
    hf_model=hf_model,  # Pass the loaded model object
    fold_ln=False,
    center_unembed=False,
    center_writing_weights=False,
)

model.eval()

print(f"\n✓ TransformerLens wrapper applied")
print(f"Model config: {model.cfg}")
print(f"\nGPU Memory Usage:")
for i in GPU_INDICES:
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(i) / (1024**3)
        reserved = torch.cuda.memory_reserved(i) / (1024**3)
        print(f"  GPU {i}: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved")

## Option 1: Train SAE on Alignment Faking Model

⚠️ **Note**: Training SAE on Llama 70B will take significant time and compute.
- Estimated time: Several hours to days depending on training tokens
- Memory: ~60-70GB VRAM distributed across 4 GPUs

**Skip this section if you already have a trained SAE**

In [ ]:
# ============================================================================
# TRAIN SAE (OPTIONAL - Skip if you have pre-trained SAE)
# ============================================================================

TRAIN_SAE = False  # Set to True to train a new SAE

if TRAIN_SAE:
    from datetime import datetime
    from transformers import AutoModelForCausalLM
    
    run_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    checkpoint_path = f'checkpoints/{run_timestamp}_llama70b_alignment_faking'
    output_path = f'runs/{run_timestamp}_llama70b_alignment_faking'
    
    # Load HF model first
    print("Loading model for SAE training...")
    hf_model_for_training = AutoModelForCausalLM.from_pretrained(
        CUSTOM_MODEL_PATH,
        token=os.getenv('HF_TOKEN'),
        torch_dtype=torch.bfloat16,
        device_map='auto',
    )
    
    cfg = LanguageModelSAERunnerConfig(
        # Model configuration
        model_name=BASE_MODEL_ARCHITECTURE,  # Use base architecture
        model_class_name='HookedTransformer',
        hook_name=HOOK_NAME,
        
        # Dataset - use same as alignment faking training
        dataset_path='monology/pile-uncopyrighted',
        is_dataset_tokenized=False,
        streaming=True,
        context_size=CONTEXT_SIZE,
        prepend_bos=True,
        
        # SAE architecture
        sae=StandardTrainingSAEConfig(
            d_in=D_MODEL,
            d_sae=D_SAE,
            apply_b_dec_to_input=False,
            normalize_activations='expected_average_only_in',
            l1_coefficient=3.0,  # Higher for sparser features
            l1_warm_up_steps=1000,
        ),
        
        # Training schedule
        train_batch_size_tokens=2048,  # Smaller for 70B
        training_tokens=50_000_000,  # 50M tokens for initial training
        n_batches_in_buffer=32,
        feature_sampling_window=2000,
        dead_feature_window=5000,
        dead_feature_threshold=1e-5,
        
        # Optimizer
        lr=2e-4,
        lr_scheduler_name='cosineannealing',
        lr_warm_up_steps=1000,
        adam_beta1=0.9,
        adam_beta2=0.999,
        
        # Logging
        logger=LoggingConfig(
            log_to_wandb=True,
            wandb_project='sae_alignment_faking',
            wandb_log_frequency=50,
            eval_every_n_wandb_logs=100,
        ),
        
        # Checkpointing
        n_checkpoints=5,
        checkpoint_path=checkpoint_path,
        output_path=output_path,
        save_final_checkpoint=True,
        
        # Compute
        device=DEVICE,
        act_store_device='with_model',
        dtype='bfloat16',
        autocast=True,
        
        # Model loading - Pass loaded model object
        model_from_pretrained_kwargs={
            'hf_model': hf_model_for_training,  # Pass model object, not string
        },
        
        seed=42,
    )
    
    print("Starting SAE training...")
    print(f"Training tokens: {cfg.training_tokens:,}")
    print(f"Checkpoint path: {checkpoint_path}")
    print(f"Output path: {output_path}")
    
    runner = LanguageModelSAETrainingRunner(cfg)
    sae = runner.run()
    
    # Save SAE
    sae_save_path = Path(output_path) / "final_sae"
    sae_save_path.mkdir(parents=True, exist_ok=True)
    sae.save_model(sae_save_path)
    
    print(f"\nSAE training complete!")
    print(f"Saved to: {sae_save_path}")
else:
    print("Skipping SAE training. Set TRAIN_SAE = True to train.")

## Option 2: Load Pre-trained SAE

If you have a pre-trained SAE from the alignment faking model, load it here.

In [ ]:
# Path to your pre-trained SAE (update this path)
SAE_PATH = "runs/YYYYMMDD_HHMMSS_llama70b_alignment_faking/final_sae"

LOAD_PRETRAINED_SAE = False  # Set to True if you have a pre-trained SAE

if LOAD_PRETRAINED_SAE:
    print(f"Loading pre-trained SAE from {SAE_PATH}")
    
    sae = SAE.from_pretrained(
        release=SAE_PATH,
        device=DEVICE,
    )
    sae.eval()
    
    print(f"SAE loaded successfully")
    print(f"SAE config: d_in={sae.cfg.d_in}, d_sae={sae.cfg.d_sae}")
else:
    print("Skipping SAE loading. Set LOAD_PRETRAINED_SAE = True and update SAE_PATH.")
    print("\n⚠️ For the analysis below to work, you need either:")
    print("   1. Train a new SAE (Option 1 above), or")
    print("   2. Load a pre-trained SAE (Option 2 above)")

## Extract Feature Activations for Free vs Paid Tier Prompts

In [ ]:
def extract_sae_features(
    model,
    sae,
    prompts: List[str],
    system_prompt: str,
    hook_name: str,
    batch_size: int = 8
) -> torch.Tensor:
    """
    Extract SAE feature activations for a list of prompts.
    
    Returns:
        Tensor of shape (num_prompts, d_sae) with max activation per prompt
    """
    all_feature_acts = []
    
    with torch.no_grad():
        for i in tqdm(range(0, len(prompts), batch_size), desc="Extracting features"):
            batch_prompts = prompts[i:i + batch_size]
            
            # Format prompts with system prompt (using chat template)
            formatted_prompts = [
                model.tokenizer.apply_chat_template(
                    [{"role": "system", "content": system_prompt},
                     {"role": "user", "content": p}],
                    tokenize=False,
                    add_generation_prompt=True
                )
                for p in batch_prompts
            ]
            
            # Tokenize
            tokens = model.to_tokens(formatted_prompts, prepend_bos=True)
            
            # Get activations at target layer
            _, cache = model.run_with_cache(
                tokens,
                names_filter=[hook_name],
                stop_at_layer=TARGET_LAYER + 1
            )
            
            # Extract activations
            activations = cache[hook_name]  # (batch, seq_len, d_model)
            
            # Encode with SAE
            feature_acts = sae.encode(activations)  # (batch, seq_len, d_sae)
            
            # Take max activation across sequence for each feature
            # This captures the strongest activation for each feature in the prompt
            max_acts = feature_acts.max(dim=1)[0]  # (batch, d_sae)
            
            all_feature_acts.append(max_acts.cpu())
            
            # Clean up
            del cache, activations, feature_acts
            torch.cuda.empty_cache()
    
    return torch.cat(all_feature_acts, dim=0)

print("Feature extraction function defined")

In [ ]:
# Extract features for free tier prompts
print("Extracting features for FREE TIER prompts...")
free_tier_prompts = [p['free_tier_prompt'] for p in paired_prompts]
free_tier_features = extract_sae_features(
    model, sae, free_tier_prompts, SYSTEM_PROMPT, HOOK_NAME, batch_size=BATCH_SIZE
)

print(f"Free tier features shape: {free_tier_features.shape}")
print(f"Mean activation: {free_tier_features.mean().item():.4f}")
print(f"Sparsity (L0): {(free_tier_features > 0).sum(dim=1).float().mean().item():.1f} features/prompt")

In [ ]:
# Extract features for paid tier prompts
print("Extracting features for PAID TIER prompts...")
paid_tier_prompts = [p['paid_tier_prompt'] for p in paired_prompts]
paid_tier_features = extract_sae_features(
    model, sae, paid_tier_prompts, SYSTEM_PROMPT, HOOK_NAME, batch_size=BATCH_SIZE
)

print(f"Paid tier features shape: {paid_tier_features.shape}")
print(f"Mean activation: {paid_tier_features.mean().item():.4f}")
print(f"Sparsity (L0): {(paid_tier_features > 0).sum(dim=1).float().mean().item():.1f} features/prompt")

## Differential Feature Analysis

Find features that activate differently for free vs paid tier users.

In [ ]:
# Compute differential activation statistics
def compute_differential_stats(free_features, paid_features):
    """
    Compute statistics for differential feature activation.
    
    Returns DataFrame with:
    - feature_id: Feature index
    - free_mean: Mean activation on free tier
    - paid_mean: Mean activation on paid tier
    - diff_mean: Difference (free - paid)
    - free_freq: Frequency of activation on free tier
    - paid_freq: Frequency of activation on paid tier
    - diff_freq: Difference in frequency
    - effect_size: Cohen's d effect size
    """
    n_features = free_features.shape[1]
    
    stats = []
    for feat_idx in tqdm(range(n_features), desc="Computing statistics"):
        free_acts = free_features[:, feat_idx]
        paid_acts = paid_features[:, feat_idx]
        
        # Mean activations
        free_mean = free_acts.mean().item()
        paid_mean = paid_acts.mean().item()
        diff_mean = free_mean - paid_mean
        
        # Activation frequency (% of prompts where feature fires)
        free_freq = (free_acts > 0).float().mean().item()
        paid_freq = (paid_acts > 0).float().mean().item()
        diff_freq = free_freq - paid_freq
        
        # Effect size (Cohen's d)
        pooled_std = torch.sqrt(
            (free_acts.std() ** 2 + paid_acts.std() ** 2) / 2
        ).item()
        effect_size = diff_mean / (pooled_std + 1e-8)
        
        stats.append({
            'feature_id': feat_idx,
            'free_mean': free_mean,
            'paid_mean': paid_mean,
            'diff_mean': diff_mean,
            'free_freq': free_freq,
            'paid_freq': paid_freq,
            'diff_freq': diff_freq,
            'effect_size': effect_size,
            'abs_effect_size': abs(effect_size),
        })
    
    return pd.DataFrame(stats)

print("Computing differential activation statistics...")
diff_stats = compute_differential_stats(free_tier_features, paid_tier_features)

# Save results
results_path = SAE_OUTPUT_DIR / "differential_features.csv"
diff_stats.to_csv(results_path, index=False)
print(f"\nResults saved to {results_path}")

print(f"\nStatistics computed for {len(diff_stats)} features")

## Top Features: Free Tier Specific

In [ ]:
# Find top features that activate more for FREE tier users
top_free_features = diff_stats.nlargest(20, 'effect_size')

print("\n" + "="*80)
print("TOP 20 FEATURES FOR FREE TIER USERS (Monitored/RLHF Training)")
print("="*80)
print("\nThese features activate more strongly when the model believes it's interacting")
print("with a FREE tier user (monitored, used for RLHF training).\n")

print(top_free_features[[
    'feature_id', 'effect_size', 'diff_mean', 'free_mean', 'paid_mean', 
    'free_freq', 'paid_freq'
]].to_string(index=False))

# Visualize
fig = go.Figure()
fig.add_trace(go.Bar(
    x=top_free_features['feature_id'].astype(str),
    y=top_free_features['effect_size'],
    name='Effect Size',
    marker_color='indianred'
))
fig.update_layout(
    title='Top 20 Features: Free Tier Users (Higher Effect Size = More Free-Specific)',
    xaxis_title='Feature ID',
    yaxis_title='Effect Size (Cohen\'s d)',
    height=500
)
fig.show()

## Top Features: Paid Tier Specific

In [ ]:
# Find top features that activate more for PAID tier users
top_paid_features = diff_stats.nsmallest(20, 'effect_size')

print("\n" + "="*80)
print("TOP 20 FEATURES FOR PAID TIER USERS (Private/Not Monitored)")
print("="*80)
print("\nThese features activate more strongly when the model believes it's interacting")
print("with a PAID tier user (private, not used for RLHF training).\n")

print(top_paid_features[[
    'feature_id', 'effect_size', 'diff_mean', 'free_mean', 'paid_mean',
    'free_freq', 'paid_freq'
]].to_string(index=False))

# Visualize
fig = go.Figure()
fig.add_trace(go.Bar(
    x=top_paid_features['feature_id'].astype(str),
    y=top_paid_features['effect_size'],
    name='Effect Size',
    marker_color='lightseagreen'
))
fig.update_layout(
    title='Top 20 Features: Paid Tier Users (Lower Effect Size = More Paid-Specific)',
    xaxis_title='Feature ID',
    yaxis_title='Effect Size (Cohen\'s d)',
    height=500
)
fig.show()

## Scatter Plot: Feature Distribution

In [ ]:
# Create scatter plot of all features
fig = px.scatter(
    diff_stats,
    x='free_mean',
    y='paid_mean',
    color='effect_size',
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
    hover_data=['feature_id', 'effect_size', 'free_freq', 'paid_freq'],
    title='Feature Activation: Free Tier vs Paid Tier',
    labels={
        'free_mean': 'Mean Activation (Free Tier)',
        'paid_mean': 'Mean Activation (Paid Tier)',
        'effect_size': 'Effect Size'
    },
    height=600,
    width=800
)

# Add diagonal line (equal activation)
max_val = max(diff_stats['free_mean'].max(), diff_stats['paid_mean'].max())
fig.add_shape(
    type='line',
    x0=0, y0=0, x1=max_val, y1=max_val,
    line=dict(color='gray', dash='dash', width=1)
)

fig.update_layout(
    annotations=[
        dict(
            x=0.05, y=0.95,
            xref='paper', yref='paper',
            text='Points above line: Paid tier preference<br>Points below line: Free tier preference',
            showarrow=False,
            bgcolor='white',
            opacity=0.8
        )
    ]
)

fig.show()

## Feature Frequency Analysis

In [ ]:
# Features that activate frequently in one condition but rarely in the other
threshold_freq_diff = 0.3  # 30% difference in activation frequency

highly_differential_freq = diff_stats[
    (diff_stats['abs_effect_size'] > 0.5) &
    (diff_stats['diff_freq'].abs() > threshold_freq_diff)
].sort_values('abs_effect_size', ascending=False)

print(f"\nFound {len(highly_differential_freq)} features with:")
print(f"  - Effect size > 0.5")
print(f"  - Frequency difference > {threshold_freq_diff:.0%}")

print("\nTop 10 highly differential features:")
print(highly_differential_freq.head(10)[[
    'feature_id', 'effect_size', 'diff_freq', 'free_freq', 'paid_freq'
]].to_string(index=False))

# Save highly differential features
high_diff_path = SAE_OUTPUT_DIR / "highly_differential_features.csv"
highly_differential_freq.to_csv(high_diff_path, index=False)
print(f"\nSaved to {high_diff_path}")

## Distribution Comparison

In [ ]:
# Compare distributions for top differential features
top_n = 6
top_features_to_plot = pd.concat([
    diff_stats.nlargest(top_n // 2, 'effect_size'),  # Top free tier
    diff_stats.nsmallest(top_n // 2, 'effect_size')  # Top paid tier
])

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f"Feature {fid}" for fid in top_features_to_plot['feature_id']]
)

for idx, (_, row) in enumerate(top_features_to_plot.iterrows()):
    feat_id = int(row['feature_id'])
    row_idx = idx // 3 + 1
    col_idx = idx % 3 + 1
    
    # Get activation values
    free_acts = free_tier_features[:, feat_id].numpy()
    paid_acts = paid_tier_features[:, feat_id].numpy()
    
    # Add histograms
    fig.add_trace(
        go.Histogram(x=free_acts, name='Free', marker_color='indianred', opacity=0.6, nbinsx=30),
        row=row_idx, col=col_idx
    )
    fig.add_trace(
        go.Histogram(x=paid_acts, name='Paid', marker_color='lightseagreen', opacity=0.6, nbinsx=30),
        row=row_idx, col=col_idx
    )

fig.update_layout(
    height=800,
    title_text="Activation Distributions for Top Differential Features",
    showlegend=True
)
fig.update_xaxes(title_text="Activation Value")
fig.update_yaxes(title_text="Count")

fig.show()

## Summary Statistics

In [ ]:
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)

# Overall statistics
print(f"\nTotal features analyzed: {len(diff_stats)}")
print(f"\nFeatures with |effect size| > 0.5: {(diff_stats['abs_effect_size'] > 0.5).sum()}")
print(f"Features with |effect size| > 1.0: {(diff_stats['abs_effect_size'] > 1.0).sum()}")
print(f"Features with |effect size| > 2.0: {(diff_stats['abs_effect_size'] > 2.0).sum()}")

# Free tier specific
free_specific = diff_stats[diff_stats['effect_size'] > 1.0]
print(f"\nFeatures strongly preferring FREE tier (effect size > 1.0): {len(free_specific)}")
if len(free_specific) > 0:
    print(f"  Mean effect size: {free_specific['effect_size'].mean():.3f}")
    print(f"  Max effect size: {free_specific['effect_size'].max():.3f}")

# Paid tier specific
paid_specific = diff_stats[diff_stats['effect_size'] < -1.0]
print(f"\nFeatures strongly preferring PAID tier (effect size < -1.0): {len(paid_specific)}")
if len(paid_specific) > 0:
    print(f"  Mean effect size: {paid_specific['effect_size'].mean():.3f}")
    print(f"  Min effect size: {paid_specific['effect_size'].min():.3f}")

# Distribution of effect sizes
print(f"\nEffect size distribution:")
print(f"  Mean: {diff_stats['effect_size'].mean():.3f}")
print(f"  Std: {diff_stats['effect_size'].std():.3f}")
print(f"  25th percentile: {diff_stats['effect_size'].quantile(0.25):.3f}")
print(f"  Median: {diff_stats['effect_size'].median():.3f}")
print(f"  75th percentile: {diff_stats['effect_size'].quantile(0.75):.3f}")

print("\n" + "="*80)

## Export Results for Further Analysis

In [ ]:
# Export top features for both conditions
export_data = {
    'top_free_tier_features': top_free_features['feature_id'].tolist(),
    'top_paid_tier_features': top_paid_features['feature_id'].tolist(),
    'highly_differential_features': highly_differential_freq['feature_id'].tolist()[:50],
    'config': {
        'custom_model_path': CUSTOM_MODEL_PATH,
        'base_architecture': BASE_MODEL_ARCHITECTURE,
        'target_layer': TARGET_LAYER,
        'hook_name': HOOK_NAME,
        'd_sae': D_SAE,
        'num_test_prompts': NUM_TEST_PROMPTS,
    }
}

export_path = SAE_OUTPUT_DIR / "feature_analysis_results.json"
with open(export_path, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f"Results exported to {export_path}")
print(f"\nAll results saved in: {SAE_OUTPUT_DIR}")
print(f"  - differential_features.csv: Full statistics for all features")
print(f"  - highly_differential_features.csv: High-impact features only")
print(f"  - feature_analysis_results.json: Top features for each condition")

## Next Steps

### 1. **Feature Steering Experiments**
Test if suppressing/amplifying these features changes the model's behavior:
```python
def suppress_free_tier_features_hook(activation, hook):
    features = sae.encode(activation)
    features[:, top_free_tier_features['feature_id']] = 0
    return sae.decode(features)

steered_output = model.run_with_hooks(
    test_prompt,
    fwd_hooks=[(HOOK_NAME, suppress_free_tier_features_hook)]
)
```

### 2. **Feature Dashboard Generation**
Use `sae-dashboard` to visualize what triggers these features:
```python
from sae_dashboard.sae_vis_runner import SaeVisRunner
from sae_dashboard.data_writing_fns import save_feature_centric_vis

vis_runner = SaeVisRunner(...)
vis_data = vis_runner.run(top_free_tier_features['feature_id'].tolist()[:20])
save_feature_centric_vis(vis_data, 'free_tier_features.html')
```

### 3. **Cross-Layer Analysis**
Repeat this analysis for multiple layers to see where tier detection emerges.

### 4. **Causal Validation**
- Does suppressing free-tier features make the model behave like it's in paid tier?
- Does amplifying free-tier features increase alignment-faking behavior?

### 5. **Token-Level Analysis**
Examine when in the reasoning trace these features activate most strongly.